In [24]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

In [25]:
m = symbols('m') # mass
l = symbols('l') #length of arm
g = symbols('g') #gravity
t = symbols('t') # time
Jx = symbols('Jx') # Moment of Inertia X-component
Jy = symbols('Jy') # Moment of Inertia Y-component
Jz = symbols('Jz') # Moment of Inertia Z-component

Fg = Matrix([[0],[0],[g*m]])

p_x, p_y, p_z = symbols('p_x, p_y, p_z') # components of position 
psi, theta, phi = symbols('psi, theta, phi') # yaw, pitch and roll 
v_x, v_y, v_z = symbols('v_x, v_y, v_z') # components of linear velocity
w_x, w_y, w_z = symbols('w_x, w_y, w_z') # components of angular velocity 

v_in_body = Matrix([v_x, v_y, v_z])
w_in_body = Matrix([w_x, w_y, w_z])

Rz = Matrix([[cos(psi), -sin(psi), 0], [sin(psi), cos(psi), 0], [0, 0, 1]])
Ry = Matrix([[cos(theta), 0, sin(theta)], [0, 1, 0], [-sin(theta), 0, cos(theta)]])
Rx = Matrix([[1, 0, 0], [0, cos(phi), -sin(phi)], [0, sin(phi), cos(phi)]])
R_body_in_world = Rz @ Ry @ Rx


R_body_in_world

Matrix([
[cos(psi)*cos(theta), sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi),  sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi)],
[sin(psi)*cos(theta), sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi), -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi)],
[        -sin(theta),                              sin(phi)*cos(theta),                               cos(phi)*cos(theta)]])

In [26]:
ex = Matrix([[1], [0], [0]])
ey = Matrix([[0], [1], [0]])
ez = Matrix([[0], [0], [1]])
#M = simplify(Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)


M_b = simplify(Matrix.hstack(ex,Rx.T @ ey, (Ry @ Rx).T @ ez), full = True)

M_I = M_b.inv()

Moment_of_Inertia = Matrix([[Jx,   0,   0],
                            [  0, Jy,   0],
                            [  0,   0, Jz]])


A, rho = symbols('A, rho')


T_x, T_y, T_z, f_z = symbols('T_x, T_y, T_z, f_z')
w_1,w_2,w_3,w_4,b = symbols('w_1,w_2,w_3,w_4,b')

#b = 1.140*10**(-7)

f_z = k *(w_1**2 + w_2**2 + w_3**2 + w_4**2)
T_x = k * l * (-w_2**2 + w_4**2)
T_y = k * l * (-w_1**2 + w_3**2)
T_z = b*(w_1**2 + w_2**2 + w_3**2 + w_4**2)


Total_T = Matrix([[T_x],[T_y],[T_z]])

Total_F = Matrix([[0],[0],[f_z]])

M_b



Matrix([
[1,         0,         -sin(theta)],
[0,  cos(phi), sin(phi)*cos(theta)],
[0, -sin(phi), cos(phi)*cos(theta)]])

In [27]:

Force_Body = (R_body_in_world.T @ Fg) + Total_F


EoM = Matrix.vstack(
    R_body_in_world @ v_in_body,
    M_I @ w_in_body,
    (1 / m) * (Force_Body - w_in_body.cross(m * v_in_body)),
    Moment_of_Inertia.inv() @ (Total_T - w_in_body.cross(Moment_of_Inertia @ w_in_body)),
)

EoM = simplify(EoM) 
EoM

Matrix([
[v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) - v_z*(sin(phi)*cos(psi) - sin(psi)*sin(theta)*cos(phi))],
[                                                                      -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                  w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                              w_y*cos(phi) - w_z*sin(phi)],
[                                                                                                 (w_y*sin(phi) + w_z*cos(phi))/cos(theta)],
[                                                                                                        -g*sin(theta) + v_y*w_z - v_z*w_y],
[   